In [12]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

dir = "/Users/shuixinjiang/Desktop/DATS6203/MidProj/training_demo/signal/datasets"
os.chdir(dir)

#bbs = np.loadtxt('bbs-train.txt')
all_info = np.loadtxt('cnn_data.txt')
label = np.loadtxt('label-train-dummy.txt')

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(all_info, label, test_size=0.3, random_state=0)

In [15]:
# input
x = tf.placeholder("float", [None, 1528])
y_ = tf.placeholder("float", [None, 2])

# inference
W = tf.Variable(tf.zeros([1528, 2]))
b = tf.Variable(tf.zeros([1, 2]))
matm=tf.matmul(x,W)
y = tf.nn.softmax(tf.matmul(x,W) + b)

# loss
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# training
train_step = tf.train.GradientDescentOptimizer(0.00001).minimize(cross_entropy)


# training cycles
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 70

for i in range(500):
    batch_start_idx = (i * batch_size) % (len(label) - batch_size)
    batch_end_idx = batch_start_idx + batch_size
    xs = x_train[batch_start_idx:batch_end_idx]
    ys = y_train[batch_start_idx:batch_end_idx]
    
    xs_test = x_test[batch_start_idx:batch_end_idx]
    ys_test = y_test[batch_start_idx:batch_end_idx]
    
    batch_xs = np.array(xs)
    batch_ys = np.array(ys)
    
    batch_xs_test = np.array(xs_test)
    batch_ys_test = np.array(ys_test)
    
    _, y_pred = sess.run((train_step, y), feed_dict={x: batch_xs, y_: batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_,axis=1)) # y=(m*c) so axis=1 along c
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    #print sess.run(correct_prediction, feed_dict={x: batch_xs, y_: batch_ys})
    #print sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys})
    
print (sess.run(accuracy, feed_dict={x: x_test, y_: y_test}))

0.659824
